## Get sizing for Stream+ Adoption Propensity for various segments

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%bigquery stream_plus_df

SELECT * FROM `cto-wln-sa-data-pr-bb5283.customer_personas_reports.TELUS_firefly_campaign_output`

## Cross Border

In [ ]:
%%bigquery cross_border_df

Select
 Distinct
 A.* ,
 If(B.imsi_num is not null, 'Cross-Border' , 'Other') as Traveler_Type

From 
    (SELECT
     Distinct
      bap_bus_bacct_num,
      subscriber_num,
      bap_billg_addr_province_cd,
      imsi_id,
     bacct_brand_id,
     bacct_bacct_typ_cd,
     If(ARRAY_LENGTH(can_us_soc)>0, 'CAN-US Plan' , 'Other') as Plan_type
     
    FROM
      `roaming-pr-66a1b0.prod_prop_scores.roaming_subs_base`
    WHERE
      prod_instnc_snapshot_dt = current_date()-2
    and bacct_brand_id = 1 and bacct_bacct_typ_cd = 'I') A
left join 
    
    `roaming-pr-66a1b0.prod_prop_scores.cross_border_VA_trips_summary` B
on 
    A.imsi_id = B.imsi_num

In [ ]:
cross_border_df.Traveler_Type.unique()

In [ ]:
cross_border_df.loc[cross_border_df.Traveler_Type=='Cross-Border']

In [ ]:
cross_border_stream_prop_df = pd.merge(cross_border_df.loc[cross_border_df.Traveler_Type=='Cross-Border'].reset_index(drop=True),
                                       stream_plus_df,
                                       how='left',
                                       left_on=['bap_bus_bacct_num', 'subscriber_num', 'imsi_id'],
                                       right_on = ['BAN', 'MSISDN', 'IMSI']
                                      )

In [ ]:
cross_border_stream_prop_df.propensity_score.fillna(0,inplace=True)

In [ ]:
cross_border_stream_prop_df.propensity_score.describe()

In [ ]:
cross_border_stream_prop_df.loc[cross_border_stream_prop_df.propensity_score == 0].shape

In [ ]:
cross_border_stream_prop_df.loc[cross_border_stream_prop_df.propensity_score > 1.447].shape

## Snowbirds

In [ ]:
%%bigquery snowbirds_df

Select
 Distinct
 A.* ,
 If(B.subscr_num is not null, 'SnowBirds' , 'Other') as Traveler_Type
 
From 
    (SELECT
     Distinct
      bap_bus_bacct_num,
      subscriber_num,
      bap_billg_addr_province_cd,
      imsi_id,
     bacct_brand_id,
     bacct_bacct_typ_cd,
     If(ARRAY_LENGTH(can_us_soc)>0, 'CAN-US Plan' , 'Other') as Plan_type
    FROM
      `roaming-pr-66a1b0.prod_prop_scores.roaming_subs_base`
    WHERE
      prod_instnc_snapshot_dt = current_date()-2
    and bacct_brand_id = 1 and bacct_bacct_typ_cd = 'I') A
left join 
    
    (SELECT
  ban,
  subscr_num,
  roamer_type,
  pp_recur_chrg_amt,
  data_usage_gb_qty,
  pi_cntrct_start_ts,
  pi_cntrct_end_ts,
  CASE
    WHEN data_usage_gb_qty < 1 THEN '00-01GB'
    WHEN data_usage_gb_qty >= 1
  AND data_usage_gb_qty < 3 THEN '01-03GB'
    WHEN data_usage_gb_qty >= 3 AND data_usage_gb_qty < 10 THEN '03-10GB'
    WHEN data_usage_gb_qty >= 10
  AND data_usage_gb_qty < 20 THEN '10-20GB'
  ELSE
  '20GB & Over'
END
  AS data_usg_bkt,
IF
  (bacct_brand_id=1,'TELUS','Koodo') AS bacct_brand_name,
  CASE
    WHEN pi_cntrct_start_ts IS NULL THEN 'No Commitment'
    WHEN pi_cntrct_end_ts < CURRENT_DATE() THEN 'Month to Month'
    WHEN pi_cntrct_end_ts BETWEEN CURRENT_DATE() AND DATE_ADD(CURRENT_DATE(),INTERVAL 3 MONTH) THEN 'Contract ends in 3 months or less'
  ELSE
  'Contract ends in 4 months or more'

END
  AS contract_lifecycle_stage,
--cp_cust_birth_dt,
IF(DATE_DIFF(CURRENT_DATE(),DATE(cp_cust_birth_dt),YEAR)>=55,TRUE,FALSE) AS  age_55_plus,
IF(cp_cust_birth_dt IS NULL,'Missing Age Info',IF(DATE_DIFF(CURRENT_DATE(),DATE(cp_cust_birth_dt),YEAR)>=55,'>=55','<55')) AS  age_group
FROM
  `roaming-pr-66a1b0.snowbirds.snowbirds_profile`
WHERE
  bacct_bacct_typ_cd ='I'
  AND pp_recur_chrg_amt>0) B
on 
    A.subscriber_num = B.subscr_num

In [ ]:
snowbirds_df.shape

In [ ]:
snowbirds_df.Traveler_Type.unique()

In [ ]:
snowbirds_stream_prop_df = pd.merge(snowbirds_df.loc[snowbirds_df.Traveler_Type=='SnowBirds'].reset_index(drop=True),
                                       stream_plus_df,
                                       how='left',
                                       left_on=['bap_bus_bacct_num', 'subscriber_num', 'imsi_id'],
                                       right_on = ['BAN', 'MSISDN', 'IMSI']
                                      )

In [ ]:
snowbirds_stream_prop_df.propensity_score.fillna(0,inplace=True)

In [ ]:
snowbirds_stream_prop_df.propensity_score.describe()

In [ ]:
snowbirds_stream_prop_df.loc[snowbirds_stream_prop_df.propensity_score == 0].shape

In [ ]:
snowbirds_stream_prop_df.loc[snowbirds_stream_prop_df.propensity_score > 1.447 ].shape